### DCGAN with Cat-Face dataset

- 아래 자료들을 참고했습니다.
    - https://towardsdatascience.com/getting-started-with-gans-using-pytorch-78e7c22a14a5
    - https://github.com/shubham7169/Projects/blob/master/CAT-dcgan.ipynb

In [ ]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image_size = 32
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) # mean, std

### Practice
- Load cat dataset

In [ ]:
transform = transforms.Compose([
    transforms.Resize(image_size), # resize image to 64. - smaller edge will be 64 -> could be 64x64, 128x64, 128x64, ... 
    transforms.CenterCrop(image_size), # if the image is larger than 64x64, crop 64x64 from center. 
    transforms.ToTensor(), # PIL Image object to Tensor
    transforms.Normalize(*stats)]) # normalize the image to have *stats mean and std

train_ds = ImageFolder('./data/', transform=transform) # automatically apply transform on loading image - (Dataset)
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)

In [ ]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0] # make the values of result tensors have range of 0~1 : normal image.

In [ ]:
def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))

In [ ]:
for images, _ in train_dl:
    show_images(images, 64) # dataset check
    break

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader(): # data = data.to(device) 를 생략할 수 있기 위함
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
print(device)

In [ ]:
train_dl = DeviceDataLoader(train_dl, device) # dataloader to default device

### Define Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
        
        nn.Conv2d(3, 16, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(16),
        nn.LeakyReLU(0.2, inplace=True),
        
        
        nn.Conv2d(16, 32, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(0.2, inplace=True),
        
        
        nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(0.2, inplace=True),
        
        nn.Conv2d(64, 1, kernel_size=4, stride=1, padding=0, bias=False),
        
        nn.Flatten(),
        nn.Sigmoid()) 
        # By applying Sigmoid, the result will have value in range of 0~1. We interpret the resulting value to be "probability"
        
    def forward(self, x):
        x = self.model(x)
        return x    

discriminator = Discriminator()
discriminator = to_device(discriminator, device)

### Define Generator

In [ ]:
latent_size = 32

class Generator(nn.Module):
    def __init__(self, latent_size):
        super(Generator, self).__init__()
        self.latent_size = latent_size
        self.model = nn.Sequential(
            
        nn.ConvTranspose2d(latent_size, 64, kernel_size=4, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        
        nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(32),
        nn.ReLU(True),
        
        nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(16),
        nn.ReLU(True),
        
        nn.ConvTranspose2d(16, 3, kernel_size=4, stride=2, padding=1, bias=False),
        nn.Tanh())
        # constantly upsamples random vector (latent) to generate image.
        # Tanh to limit output value range.
    def forward(self, x):
        x = self.model(x)
        return x  
    
generator = Generator(latent_size)

### Initial Generator Results

In [ ]:
xb = torch.randn(batch_size, latent_size, 1, 1) # random latent tensors
fake_images = generator(xb)
print(fake_images.shape)
show_images(fake_images)
# Generator is not trained. 
# It outputs random noise.

In [ ]:
generator = to_device(generator, device)

### Practice
- Implement discriminator loss


$$ \mathcal{L}_{D} = \mathbb{E}_{z \sim N(0,I), x \sim p_{data}(x)}[log(D(x)) + log(1 - D(G(z)))] $$
$$  $$
$$ or $$
$$ \mathcal{L}_{D} = \frac{1}{N}\sum_{i}^{N}[log(D(x_{i})) + log(1 - D(G(z_{i})))] $$

In [ ]:
def train_discriminator(real_images, opt_d):
    # Clear discriminator gradients
    opt_d.zero_grad()

    # Pass real images through discriminator
    real_preds = discriminator(real_images)
    real_targets = torch.ones(real_images.size(0), 1, device=device)
    real_loss = F.binary_cross_entropy(real_preds, real_targets)
    real_score = torch.mean(real_preds).item()
    
    # Generate fake images
    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_images = generator(latent)

    # Pass fake images through discriminator
    fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
    fake_preds = discriminator(fake_images)
    fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)
    fake_score = torch.mean(fake_preds).item()

    # Update discriminator weights
    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()
    return loss.item(), real_score, fake_score

### Practice
- Implement generator loss

$$ \mathcal{L}_{G} = \mathbb{E}_{z \sim N(0,I)}[log(1 - D(G(z)))] $$
$$  $$
$$ or $$
$$ \mathcal{L}_{G} = \frac{1}{N} \sum_{i}^{N}[log(1 - D(G(z_{i})))] $$

In [ ]:
def train_generator(opt_g):
    # Clear generator gradients
    opt_g.zero_grad()
    
    # Generate fake images
    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_images = generator(latent)
    
    # Try to fool the discriminator
    preds = discriminator(fake_images)
    targets = torch.ones(batch_size, 1, device=device)
    loss = F.binary_cross_entropy(preds, targets)
    
    # Update generator weights
    loss.backward()
    opt_g.step()
    
    return loss.item()

In [ ]:
sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)

In [ ]:
def save_samples(index, latent_tensors, show=True):
    fake_images = generator(latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(denorm(fake_images.cpu().detach()), nrow=8).permute(1, 2, 0))

In [ ]:
fixed_latent = torch.randn(64, latent_size, 1, 1, device=device) # constant latent vector -> to see learning process

In [ ]:
save_samples(0, fixed_latent)

In [ ]:
from IPython.display import Image

In [ ]:
def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()
    
    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
    
    # Create optimizers
    opt_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        generator.train()
        discriminator.train()
        for real_images, _ in tqdm(train_dl):
            # Train discriminator
            loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
            # Train generator
            loss_g = train_generator(opt_g)
            
        # Record losses & scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
    
        # Save generated images
        with torch.no_grad():
            generator.eval()
            save_samples(epoch+start_idx, fixed_latent, show=True)
            fake_images = generator(fixed_latent)
            plt.clf()
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.set_xticks([]); ax.set_yticks([])
            ax.imshow(make_grid(denorm(fake_images.cpu().detach()), nrow=8).permute(1, 2, 0))
            plt.show()
        
    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
lr = 0.0002
epochs = 60

In [ ]:
history = fit(epochs, lr)

In [ ]:
losses_g, losses_d, real_scores, fake_scores = history

In [ ]:
Image('./generated/generated-images-0060.png')


In [ ]:
plt.plot(losses_d, '-')
plt.plot(losses_g, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real', 'Fake'])
plt.title('Scores');

In [ ]:
torch.save(generator.state_dict(), 'generator.pkl')
torch.save(discriminator.state_dict(), 'discriminator.pkl')